### Refine the cobra motor maps

In [14]:
from importlib import reload
import numpy as np
import time
from astropy.io import fits
import sep
import matplotlib.pyplot as plt
from subprocess import Popen, PIPE
import glob
from copy import deepcopy
from ics.cobraCharmer import pfi as pfiControl

In [7]:
mod1Cobras = pfiControl.PFI.allocateCobraRange(range(1,2))
allCobras = mod1Cobras
oneCobra = pfiControl.PFI.allocateCobraList([(1,2)])
twoCobras = pfiControl.PFI.allocateCobraList([(1,2), (1,5)])

# partition module 1 cobras into non-interfering sets
moduleCobras = {}
for group in 1,2,3:
    cm = range(group,58,3)
    mod = [1]*len(cm)
    moduleCobras[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
group1Cobras = moduleCobras[1]
group2Cobras = moduleCobras[2]
group3Cobras = moduleCobras[3]

# partition module 1 cobras into odd and even sets
moduleCobras2 = {}
for group in 1,2:
    cm = range(group,58,2)
    mod = [1]*len(cm)
    moduleCobras2[group] = pfiControl.PFI.allocateCobraList(zip(mod,cm))
oddCobras = moduleCobras2[1]
evenCobras = moduleCobras2[2]

In [16]:
#pfi = pfiControl.PFI(fpgaHost='128.149.77.24') #'fpga' for real device.
pfi = pfiControl.PFI(fpgaHost='localhost', doLoadModel=False)
pfi.loadModel('/Users/chyan/Documents/workspace/ics_cobraCharmer/xml/updatedLinksAndMaps.xml')
pfi.setFreq(allCobras)

2018-12-04T13:54:32.997 20 fpgaIO     FPGA connection to localhost
2018-12-04T13:54:33.058 20 fpga       load cobra model from /Users/chyan/Documents/workspace/ics_cobraCharmer/xml/updatedLinksAndMaps.xml
2018-12-04T13:54:33.059 20 log.short  --- ISSUE SETFREQ & VERIFY (brd:1) ---
2018-12-04T13:54:33.061 20 log.eth    (ETH)Sent msg on socket.
(0300003907d088cd	081300fd00940823	00fd0099101300fb	009a102300fc0099	
181300fd00981823	00fc009720130100	0097202300fc0098	281300fa00992823	
00fc0097301300fe	0095302300fd0098	381300fe00973823	00fc0098401300fa	
0097402300fc009a	481300fc00984823	00fe0096501300fd	0098502300fa0097	
581300fb00975823	00fe0099601300fc	0099602300fa0098	681300fd00976823	
01000096701300fd	0099702300fd0094	781300fc00977823	00fc0098801300fb	
0093802300fb0096	881300fb00988823	00fd0098901300fd	0096902300fa0098	
981300ff00979823	00fa0098a01300ff	0099a02300fe0096	a81300fc0097a823	
01000098b01300fd	0097b02301010099	b81300fa0097b823	00fc0098c01300fa	
0097c02300fe0098	c81300fc0096c823

In [10]:
pfi.loadModel('../xml/precise.xml')
pfi.setFreq(allCobras)

NameError: name 'pfi' is not defined

In [ ]:
# Calculate up/down(outward) angles
oddMoves = pfi.thetaToLocal(oddCobras, [np.deg2rad(270)]*len(oddCobras))
oddMoves[oddMoves>1.85*np.pi] = 0

evenMoves = pfi.thetaToLocal(evenCobras, [np.deg2rad(90)]*len(evenCobras))
evenMoves[evenMoves>1.85*np.pi] = 0

allMoves = np.zeros(57)
allMoves[::2] = oddMoves
allMoves[1::2] = evenMoves

allSteps, _ = pfi.calculateSteps(np.zeros(57), allMoves, np.zeros(57), np.zeros(57))

def moveCobra(c, theta, phi):
    pfi.moveSteps([allCobras[c-1]], np.zeros(1)+theta, np.zeros(1)+phi)

def moveCobras(cs, theta, phi):
    cobs = []
    for c in cs:
        cobs.append(allCobras[c-1])
    pfi.moveSteps(cobs, np.array(theta), np.array(phi))

In [12]:
# define the broken/good cobras
brokens = [1, 39, 43, 54]
visibles= [e for e in range(1,58) if e not in brokens]
badIdx = np.array(brokens) - 1
goodIdx = np.array(visibles) - 1

# two groups for two cameras
cam_split = 26
group1 = goodIdx[goodIdx <= cam_split]
group2 = goodIdx[goodIdx > cam_split]

# three non-interfering groups for good cobras
goodGroupIdx = {}
for group in range(3):
    goodGroupIdx[group] = goodIdx[goodIdx%3==group]

def getCobras(cobs):
    # cobs is 0-indexed list
    return pfiControl.PFI.allocateCobraList(zip(np.full(len(cobs), 1), np.array(cobs) + 1))

def thetaFN(camId, group):
    return f'data/theta{camId}G{group}_'

def phiFN(camId, group):
    return f'data/phi{camId}G{group}_'

In [ ]:
# Home phi
pfi.moveAllSteps(allCobras, 0, -5000)

In [ ]:
# Home theta
pfi.moveAllSteps(allCobras, -10000, 0)

In [ ]:
# Move the bad cobras to up/down positions
pfi.moveSteps(getCobras(badIdx), allSteps[badIdx], np.zeros(len(brokens)))

In [ ]:
# Manually fine tuning the bad cobras
moveCobra(54, -200, 0)

In [18]:
def lazyIdentification(centers, spots, radii=None):
    n = len(centers)
    if radii is not None and len(radii) != n:
        raise RuntimeError("number of centers must match number of radii")
    ans = np.empty(n, dtype=int)
    for i in range(n):
        dist = np.absolute(spots - centers[i])
        j = np.argmin(dist)
        if radii is not None and np.absolute(centers[i] - spots[j]) > radii[i]:
            ans[i] = -1
        else:
            ans[i] = j
    return ans

In [19]:
# function to move cobras to target positions
def moveToXYfromHome(idx, targets, threshold=3.0, maxTries=8):
    cobras = getCobras(idx)
    pfi.moveXYfromHome(cobras, targets)

    ntries = 1
    while True:
        # check current positions, first exposing
        p1 = Popen(["/home/pfs/IDSControl/idsexposure", "-d", "1", "-e", "18", "-f", "data/cam1_"], stdout=PIPE)
        p1.communicate()
        p2 = Popen(["/home/pfs/IDSControl/idsexposure", "-d", "2", "-e", "18", "-f", "data/cam2_"], stdout=PIPE)
        p2.communicate()

        # extract sources and fiber identification
        data1 = fits.getdata('data/cam1_0001.fits').astype(float)
        ext1 = sep.extract(data1, 100)
        idx1 = lazyIdentification(pfi.calibModel.centers[idx[idx <= cam_split]], ext1['x'] + ext1['y']*(1j))
        data2 = fits.getdata('data/cam2_0001.fits').astype(float)
        ext2 = sep.extract(data2, 100)
        idx2 = lazyIdentification(pfi.calibModel.centers[idx[idx > cam_split]], ext2['x'] + ext2['y']*(1j))
        curPos = np.concatenate((ext1[idx1]['x'] + ext1[idx1]['y']*(1j), ext2[idx2]['x'] + ext2[idx2]['y']*(1j)))
        print(curPos)

        # check position errors
        done = np.abs(curPos - targets) <= threshold
        if np.all(done):
            print('Convergence sequence done')
            break
        if ntries > maxTries:
            print(f'Reach max {maxTries} tries, gave up')
            break
        ntries += 1

        # move again
        pfi.moveXY(cobras, curPos, targets)

In [ ]:
# move visible positioners to outwards positions, phi arms are moved out for 60 degrees
# (outTargets) otherwise we can't measure the theta angles
thetas = np.empty(57, dtype=float)
thetas[::2] = pfi.thetaToLocal(oddCobras, np.full(len(oddCobras), np.deg2rad(270)))
thetas[1::2] = pfi.thetaToLocal(evenCobras, np.full(len(evenCobras), np.deg2rad(90)))
phis = np.full(57, np.deg2rad(60.0))
outTargets = pfi.anglesToPositions(allCobras, thetas, phis)

In [ ]:
# move to outTargets
moveToXYfromHome(goodIdx, outTargets[goodIdx])

# move phi arms in
pfi.moveAllSteps(getCobras(goodIdx), 0, -5000)

In [ ]:
# You may want to check if all cobras are really pointing outward
moveCobra(41, 200, 0)

In [ ]:
def expose(fn1, fn2):
    p1 = Popen(["/home/pfs/IDSControl/idsexposure", "-d", "1", "-e", "18", "-l", "3", "-f", fn1], stdout=PIPE)
    p1.communicate()
    p2 = Popen(["/home/pfs/IDSControl/idsexposure", "-d", "2", "-e", "18", "-l", "3", "-f", fn2], stdout=PIPE)
    p2.communicate()

In [21]:
# parameters declared here
repeat = 3
steps = 400
thetaSteps = 10000
phiSteps = 5000
myCobras = getCobras(goodIdx)

In [ ]:
# record the phi movements
for n in range(repeat):
    # forward phi motor maps
    expose(f'data/phi1Begin{n}_', f'data/phi2Begin{n}_')
    for k in range(phiSteps//steps):
        pfi.moveAllSteps(myCobras, 0, steps)
        expose(f'data/phi1Forward{n}N{k}_', f'data/phi2Forward{n}N{k}_')
    # reverse phi motor maps
    expose(f'data/phi1End{n}_', f'data/phi2End{n}_')
    for k in range(phiSteps//steps):
        pfi.moveAllSteps(myCobras, 0, -steps)
        expose(f'data/phi1Reverse{n}N{k}_', f'data/phi2Reverse{n}N{k}_')

# move phi arms out for 60 degrees then home theta
pfi.moveAllSteps(myCobras, 0, -5000)
moveToXYfromHome(goodIdx, outTargets[goodIdx])
pfi.moveAllSteps(myCobras, -10000, 0)

# record the theta movements
for n in range(repeat):
    # forward theta motor maps
    expose(f'data/theta1Begin{n}_', f'data/theta2Begin{n}_')
    for k in range(thetaSteps//steps):
        pfi.moveAllSteps(myCobras, steps, 0)
        expose(f'data/theta1Forward{n}N{k}_', f'data/theta2Forward{n}N{k}_')
    # reverse theta motor maps
    expose(f'data/theta1End{n}_', f'data/theta2End{n}_')
    for k in range(thetaSteps//steps):
        pfi.moveAllSteps(myCobras, -steps, 0)
        expose(f'data/theta1Reverse{n}N{k}_', f'data/theta2Reverse{n}N{k}_')

In [22]:
# variable declaration for position measurement
thetaFW = np.zeros((57, repeat, thetaSteps//steps+1), dtype=complex)
thetaRV = np.zeros((57, repeat, thetaSteps//steps+1), dtype=complex)
phiFW = np.zeros((57, repeat, phiSteps//steps+1), dtype=complex)
phiRV = np.zeros((57, repeat, phiSteps//steps+1), dtype=complex)

In [24]:
# first camera

# phi stages
myIdx = goodIdx[goodIdx <= cam_split]
centers = pfi.calibModel.centers[myIdx]

# forward phi
cnt = phiSteps//steps
for n in range(repeat):
    data = fits.getdata(f'/Volumes/Science/20181128/phi2Begin{n}_0001.fits')
    cs = sep.extract(data.astype(float), 50)
    spots = np.array([c['x']+c['y']*(1j) for c in cs])
    idx = lazyIdentification(centers, spots)
    phiFW[myIdx,n,0] = spots[idx]   
    for k in range(cnt):
        data = fits.getdata(f'/Volumes/Science/20181128/phi2Forward{n}N{k}_0001.fits')
        cs = sep.extract(data.astype(float), 50)
        spots = np.array([c['x']+c['y']*(1j) for c in cs])
        idx = lazyIdentification(centers, spots)
        phiFW[myIdx,n,k+1] = spots[idx]

# reverse phi
for n in range(repeat):
    data = fits.getdata(f'/Volumes/Science/20181128/phi2End{n}_0001.fits')
    cs = sep.extract(data.astype(float), 50)
    spots = np.array([c['x']+c['y']*(1j) for c in cs])
    idx = lazyIdentification(centers, spots)
    phiRV[myIdx,n,0] = spots[idx]   
    for k in range(cnt):
        data = fits.getdata(f'/Volumes/Science/20181128/phi2Reverse{n}N{k}_0001.fits')
        cs = sep.extract(data.astype(float), 50)
        spots = np.array([c['x']+c['y']*(1j) for c in cs])
        idx = lazyIdentification(centers, spots)
        phiRV[myIdx,n,k+1] = spots[idx]

# forward theta
cnt = thetaSteps//steps
for n in range(repeat):
    data = fits.getdata(f'/Volumes/Science/20181128/theta2Begin{n}_0001.fits')
    cs = sep.extract(data.astype(float), 50)
    spots = np.array([c['x']+c['y']*(1j) for c in cs])
    idx = lazyIdentification(centers, spots)
    thetaFW[myIdx,n,0] = spots[idx]   
    for k in range(cnt):
        data = fits.getdata(f'/Volumes/Science/20181128/theta2Forward{n}N{k}_0001.fits')
        cs = sep.extract(data.astype(float), 50)
        spots = np.array([c['x']+c['y']*(1j) for c in cs])
        idx = lazyIdentification(centers, spots)
        thetaFW[myIdx,n,k+1] = spots[idx]

# reverse theta
for n in range(repeat):
    data = fits.getdata(f'/Volumes/Science/20181128/theta2End{n}_0001.fits')
    cs = sep.extract(data.astype(float), 50)
    spots = np.array([c['x']+c['y']*(1j) for c in cs])
    idx = lazyIdentification(centers, spots)
    thetaRV[myIdx,n,0] = spots[idx]   
    for k in range(cnt):
        data = fits.getdata(f'/Volumes/Science/20181128/theta2Reverse{n}N{k}_0001.fits')
        cs = sep.extract(data.astype(float), 50)
        spots = np.array([c['x']+c['y']*(1j) for c in cs])
        idx = lazyIdentification(centers, spots)
        thetaRV[myIdx,n,k+1] = spots[idx]

In [25]:
# second camera

# phi stages
myIdx = goodIdx[goodIdx > cam_split]
centers = pfi.calibModel.centers[myIdx]

# forward phi
cnt = phiSteps//steps
for n in range(repeat):
    data = fits.getdata(f'/Volumes/Science/20181128/phi1Begin{n}_0001.fits')
    cs = sep.extract(data.astype(float), 50)
    spots = np.array([c['x']+c['y']*(1j) for c in cs])
    idx = lazyIdentification(centers, spots)
    phiFW[myIdx,n,0] = spots[idx]   
    for k in range(cnt):
        data = fits.getdata(f'/Volumes/Science/20181128/phi1Forward{n}N{k}_0001.fits')
        cs = sep.extract(data.astype(float), 50)
        spots = np.array([c['x']+c['y']*(1j) for c in cs])
        idx = lazyIdentification(centers, spots)
        phiFW[myIdx,n,k+1] = spots[idx]

# reverse phi
for n in range(repeat):
    data = fits.getdata(f'/Volumes/Science/20181128/phi1End{n}_0001.fits')
    cs = sep.extract(data.astype(float), 50)
    spots = np.array([c['x']+c['y']*(1j) for c in cs])
    idx = lazyIdentification(centers, spots)
    phiRV[myIdx,n,0] = spots[idx]   
    for k in range(cnt):
        data = fits.getdata(f'/Volumes/Science/20181128/phi1Reverse{n}N{k}_0001.fits')
        cs = sep.extract(data.astype(float), 50)
        spots = np.array([c['x']+c['y']*(1j) for c in cs])
        idx = lazyIdentification(centers, spots)
        phiRV[myIdx,n,k+1] = spots[idx]

# forward theta
cnt = thetaSteps//steps
for n in range(repeat):
    data = fits.getdata(f'/Volumes/Science/20181128/theta1Begin{n}_0001.fits')
    cs = sep.extract(data.astype(float), 50)
    spots = np.array([c['x']+c['y']*(1j) for c in cs])
    idx = lazyIdentification(centers, spots)
    thetaFW[myIdx,n,0] = spots[idx]   
    for k in range(cnt):
        data = fits.getdata(f'/Volumes/Science/20181128/theta1Forward{n}N{k}_0001.fits')
        cs = sep.extract(data.astype(float), 50)
        spots = np.array([c['x']+c['y']*(1j) for c in cs])
        idx = lazyIdentification(centers, spots)
        thetaFW[myIdx,n,k+1] = spots[idx]

# reverse theta
for n in range(repeat):
    data = fits.getdata(f'/Volumes/Science/20181128/theta1End{n}_0001.fits')
    cs = sep.extract(data.astype(float), 50)
    spots = np.array([c['x']+c['y']*(1j) for c in cs])
    idx = lazyIdentification(centers, spots)
    thetaRV[myIdx,n,0] = spots[idx]   
    for k in range(cnt):
        data = fits.getdata(f'/Volumes/Science/20181128/theta1Reverse{n}N{k}_0001.fits')
        cs = sep.extract(data.astype(float), 50)
        spots = np.array([c['x']+c['y']*(1j) for c in cs])
        idx = lazyIdentification(centers, spots)
        thetaRV[myIdx,n,k+1] = spots[idx]

In [26]:
# variable declaration for theta, phi angles
thetaCenter = np.zeros(57, dtype=complex)
phiCenter = np.zeros(57, dtype=complex)
thetaAngFW = np.zeros((57, repeat, thetaSteps//steps+1), dtype=float)
thetaAngRV = np.zeros((57, repeat, thetaSteps//steps+1), dtype=float)
phiAngFW = np.zeros((57, repeat, phiSteps//steps+1), dtype=float)
phiAngRV = np.zeros((57, repeat, phiSteps//steps+1), dtype=float)

In [27]:
def circle_fitting(p):
    x = np.real(p)
    y = np.imag(p)
    m = np.vstack([x, y, np.ones(len(p))]).T
    n = np.array(x*x + y*y)
    a, b, c = np.linalg.lstsq(m, n, rcond=None)[0]
    return a/2, b/2, np.sqrt(c+(a*a+b*b)/4)

In [28]:
# measure centers
for c in goodIdx:
    data = np.concatenate((thetaFW[c].flatten(), thetaRV[c].flatten()))
    x, y, r = circle_fitting(data)
    thetaCenter[c] = x + y*(1j)
    data = np.concatenate((phiFW[c].flatten(), phiRV[c].flatten()))
    x, y, r = circle_fitting(data)
    phiCenter[c] = x + y*(1j)

In [29]:
# measure theta angles
cnt = thetaSteps//steps
for c in goodIdx:
    for n in range(repeat):
        for k in range(cnt+1):
            thetaAngFW[c,n,k] = np.angle(thetaFW[c,n,k] - thetaCenter[c])
            thetaAngRV[c,n,k] = np.angle(thetaRV[c,n,k] - thetaCenter[c])
        home = thetaAngFW[c,n,0]
        thetaAngFW[c,n] = (thetaAngFW[c,n] - home) % (np.pi*2)
        thetaAngRV[c,n] = (thetaAngRV[c,n] - home) % (np.pi*2)

# fix over 2*pi angle issue
for c in goodIdx:
    for n in range(repeat):
        for k in range(cnt):
            if thetaAngFW[c,n,k+1] < thetaAngFW[c,n,k]:
                thetaAngFW[c,n,k+1] += np.pi*2
        for k in range(cnt):
            if thetaAngRV[c,n,k+1] > thetaAngRV[c,n,k]:
                thetaAngRV[c,n,k] += np.pi*2
            else:
                break
        for k in range(cnt):
            if thetaAngRV[c,n,k+1] > thetaAngRV[c,n,k]:
                thetaAngRV[c,n,k+1] -= np.pi*2

# measure phi angles
cnt = phiSteps//steps + 1
for c in goodIdx:
    for n in range(repeat):
        for k in range(cnt):
            phiAngFW[c,n,k] = np.angle(phiFW[c,n,k] - phiCenter[c])
            phiAngRV[c,n,k] = np.angle(phiRV[c,n,k] - phiCenter[c])
        home = phiAngFW[c,n,0]
        phiAngFW[c,n] = (phiAngFW[c,n] - home + np.pi/2) % (np.pi*2) - np.pi/2
        phiAngRV[c,n] = (phiAngRV[c,n] - home + np.pi/2) % (np.pi*2) - np.pi/2

In [30]:
# use the same weighting as Johannes to calculate motor maps
binSize = np.deg2rad(3.6)
regions = 112

thetaMMFW = np.zeros((57, regions), dtype=float)
thetaMMRV = np.zeros((57, regions), dtype=float)
phiMMFW = np.zeros((57, regions), dtype=float)
phiMMRV = np.zeros((57, regions), dtype=float)

In [31]:
delta = np.deg2rad(10)
thetaHS = np.deg2rad(370)

# calculate theta motor maps
cnt = thetaSteps//steps
for c in goodIdx:
    for b in range(regions):
        # forward motor maps
        binMin = binSize * b
        binMax = binMin + binSize
        fracSum = 0
        valueSum = 0
        for n in range(repeat):
            for k in range(cnt):
                if thetaAngFW[c,n,k] < binMax and thetaAngFW[c,n,k+1] > binMin and thetaAngFW[c,n,k+1] <= thetaHS:
                    moveSizeInBin = np.min([thetaAngFW[c,n,k+1], binMax]) - np.max([thetaAngFW[c,n,k], binMin])
                    entireMoveSize = thetaAngFW[c,n,k+1] - thetaAngFW[c,n,k]
                    fraction = moveSizeInBin * moveSizeInBin / entireMoveSize
                    fracSum += fraction
                    valueSum += fraction * entireMoveSize / steps
        if fracSum > 0:
            thetaMMFW[c,b] = valueSum / fracSum
        else:
            thetaMMFW[c,b] = thetaMMFW[c,b-1]

        # reverse motor maps
        fracSum = 0
        valueSum = 0
        for n in range(repeat):
            for k in range(cnt):
                if thetaAngRV[c,n,k] > binMin and thetaAngRV[c,n,k+1] < binMax and thetaAngFW[c,n,k+1] >= delta:
                    moveSizeInBin = np.min([thetaAngRV[c,n,k], binMax]) - np.max([thetaAngRV[c,n,k+1], binMin])
                    entireMoveSize = thetaAngRV[c,n,k] - thetaAngRV[c,n,k+1]
                    fraction = moveSizeInBin * moveSizeInBin / entireMoveSize
                    fracSum += fraction
                    valueSum += fraction * entireMoveSize / steps
        if fracSum > 0:
            thetaMMRV[c,b] = valueSum / fracSum
        else:
            thetaMMRV[c,b] = thetaMMFW[c,b-1]

# calculate phi motor maps
cnt = phiSteps//steps
for c in goodIdx:
    for b in range(regions):
        # forward motor maps
        binMin = binSize * b
        binMax = binMin + binSize
        fracSum = 0
        valueSum = 0
        for n in range(repeat):
            for k in range(cnt):
                if phiAngFW[c,n,k] < binMax and phiAngFW[c,n,k+1] > binMin and phiAngFW[c,n,k+1] <= np.pi - delta:
                    moveSizeInBin = np.min([phiAngFW[c,n,k+1], binMax]) - np.max([phiAngFW[c,n,k], binMin])
                    entireMoveSize = phiAngFW[c,n,k+1] - phiAngFW[c,n,k]
                    fraction = moveSizeInBin * moveSizeInBin / entireMoveSize
                    fracSum += fraction
                    valueSum += fraction * entireMoveSize / steps
        if fracSum > 0:
            phiMMFW[c,b] = valueSum / fracSum
        else:
            phiMMFW[c,b] = phiMMFW[c,b-1]

        # reverse motor maps
        fracSum = 0
        valueSum = 0
        for n in range(repeat):
            for k in range(cnt):
                if phiAngRV[c,n,k] > binMin and phiAngRV[c,n,k+1] < binMax and phiAngFW[c,n,k+1] >= delta:
                    moveSizeInBin = np.min([phiAngRV[c,n,k], binMax]) - np.max([phiAngRV[c,n,k+1], binMin])
                    entireMoveSize = phiAngRV[c,n,k] - phiAngRV[c,n,k+1]
                    fraction = moveSizeInBin * moveSizeInBin / entireMoveSize
                    fracSum += fraction
                    valueSum += fraction * entireMoveSize / steps
        if fracSum > 0:
            phiMMRV[c,b] = valueSum / fracSum
        else:
            phiMMRV[c,b] = phiMMFW[c,b-1]

In [32]:
# save new configuration for both slow nad fast motor maps
old = pfi.calibModel

sThetaFW = binSize / old.S1Pm
sThetaRV = binSize / old.S1Nm
fThetaFW = binSize / old.F1Pm
fThetaRV = binSize / old.F1Nm
sPhiFW = binSize / old.S2Pm
sPhiRV = binSize / old.S2Nm
fPhiFW = binSize / old.F2Pm
fPhiRV = binSize / old.F2Nm

# you can remove bad measurement here
idx = np.delete(goodIdx, np.argwhere(goodIdx==11))
#idx = goodIdx

sThetaFW[idx] = thetaMMFW[idx]
sThetaRV[idx] = thetaMMRV[idx]
fThetaFW[idx] = thetaMMFW[idx]
fThetaRV[idx] = thetaMMRV[idx]
sPhiFW[idx] = phiMMFW[idx]
sPhiRV[idx] = phiMMRV[idx]
fPhiFW[idx] = phiMMFW[idx]
fPhiRV[idx] = phiMMRV[idx]

# update configuration
old.updateMotorMaps(sThetaFW, sThetaRV, sPhiFW, sPhiRV, useSlowMaps=True)
old.updateMotorMaps(fThetaFW, fThetaRV, fPhiFW, fPhiRV, useSlowMaps=False)

# write to a new XML file
old.createCalibrationFile('/Users/chyan/Documents/workspace/ics_cobraCharmer/xml/motormaps.xml')

/Users/chyan/Documents/workspace/ics_cobraCharmer/python/ics/cobraCharmer/pfiDesign.py:220: RuntimeWarning: divide by zero encountered in true_divide
  self.S2Nm[i] = self.angularSteps[i] / phiRev[i]
/Users/chyan/Documents/workspace/ics_cobraCharmer/python/ics/cobraCharmer/pfiDesign.py:212: RuntimeWarning: divide by zero encountered in true_divide
  self.S2Pm[i] = self.angularSteps[i] / phiFwd[i]
/Users/chyan/Documents/workspace/ics_cobraCharmer/python/ics/cobraCharmer/pfiDesign.py:204: RuntimeWarning: divide by zero encountered in true_divide
  self.S1Nm[i] = self.angularSteps[i] / thtRev[i]
/Users/chyan/Documents/workspace/ics_cobraCharmer/python/ics/cobraCharmer/pfiDesign.py:222: RuntimeWarning: divide by zero encountered in true_divide
  self.F2Nm[i] = self.angularSteps[i] / phiRev[i]
/Users/chyan/Documents/workspace/ics_cobraCharmer/python/ics/cobraCharmer/pfiDesign.py:214: RuntimeWarning: divide by zero encountered in true_divide
  self.F2Pm[i] = self.angularSteps[i] / phiFwd[i]


In [ ]:
[(c.module, c.cobraNum) for c in getCobras(np.array([1,2,3]))]

In [ ]:
plt.plot(thetaMMFW[1]), plt.plot(thetaMMRV[1])

In [ ]:
plt.plot(phiMMFW[1]), plt.plot(phiMMRV[1])